In [1]:
from khaiii import KhaiiiApi

In [2]:
kakao_stemmer = KhaiiiApi()

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('./data/news.csv')
df = df[df['category']==1]

## Clean up Sentence

In [5]:
import re

def preprocessing(sentence):
    sentence =re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', sentence)
    return sentence

In [6]:
df['content_cleaned'] = df['content'].apply(preprocessing)
content = df['content_cleaned'].tolist()

In [7]:
len(content)

2872

## Tokenization

In [8]:
result = kakao_stemmer.analyze(content[0])
word_tokens = [l.lex for mor in result for l in mor.morphs if l.tag == 'NNG']
word_tokens[:5]

['학년도', '수시', '정시', '통합', '방안']

In [9]:
def tokenize(sentence):
    result = kakao_stemmer.analyze(sentence)
    word_tokens = [l.lex for mor in result for l in mor.morphs if l.tag == 'NNG']
    ret = ' '.join(word_tokens)
    return ret

In [10]:
df['content_tokenized'] = df['content_cleaned'].apply(tokenize)

## Count Vectorizer

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
# vectorizing
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['content_tokenized'])

In [13]:
vectorizer.vocabulary_

{'학년도': 20414,
 '수시': 10016,
 '정시': 16086,
 '통합': 19395,
 '방안': 6867,
 '수능': 9933,
 '무력': 5994,
 '대입': 4299,
 '제도': 16206,
 '학생': 20461,
 '부종합전': 7790,
 '학종': 20566,
 '꼼수': 3193,
 '불신': 8006,
 '현실': 21160,
 '외면': 12854,
 '혼란': 21367,
 '교수': 1925,
 '국가': 2276,
 '교육': 1967,
 '공정': 1498,
 '사회를위': 8536,
 '모임': 5867,
 '사회': 8527,
 '오전': 12674,
 '기자': 3040,
 '회견': 21654,
 '교육회': 2072,
 '위원': 13367,
 '문학': 6152,
 '사퇴': 8513,
 '촉구': 18229,
 '옹호': 12780,
 '개편': 582,
 '걸림돌': 722,
 '주장': 16702,
 '논란': 3615,
 '확산': 21516,
 '전날': 15660,
 '오송': 12658,
 '고교': 1122,
 '학점제': 20562,
 '연구': 12167,
 '학교': 20361,
 '교원': 1961,
 '연수': 12270,
 '행사': 20975,
 '미래': 6323,
 '전형': 15883,
 '주제': 16709,
 '발표': 6785,
 '불씨': 8007,
 '고등': 1152,
 '교육전문': 2039,
 '위원회': 13372,
 '참여': 17653,
 '자료': 14927,
 '성적': 9468,
 '학생부': 20465,
 '면접': 5700,
 '평가': 19840,
 '제안': 16243,
 '과목': 1569,
 '절대평가': 15902,
 '연계': 12162,
 '폐지': 19931,
 '현행': 21190,
 '시기': 10520,
 '구분': 2187,
 '원서': 13210,
 '접수': 15959,
 '준비': 16764,
 '학기': 20405

## Tfidf Vectorizer

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [15]:
tfidf_matrix = tfidf.fit_transform(df['content_tokenized'])

In [16]:
print(tfidf_matrix.shape)

(2872, 21967)


## Cosine 유사도

In [17]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [18]:
cosine_sim[0]

array([1.        , 0.04014056, 0.03967585, ..., 0.01330341, 0.00925404,
       0.06885469])

In [19]:
def get_recommendations(idx, cosine_sim=cosine_sim):
    # 모든 뉴스에 대해서 해당 뉴스와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 뉴스들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 뉴스를 받아옵니다.
    sim_scores = sim_scores[1:11]

    ret = [(i) for i in sim_scores]
    return ret

In [31]:
def top_10_similarity(idx):
    print(idx)
    ret = get_recommendations(idx)
    print(ret)
    return (idx, ret)

In [49]:
ret = top_10_similarity(24)

24
[(702, 1.0000000000000002), (118, 0.36316327926267294), (469, 0.27300724869413373), (823, 0.2645829319758984), (431, 0.25693443393448245), (507, 0.2514053420934368), (1528, 0.20705041818559763), (1956, 0.20385908032688546), (47, 0.2006747343758437), (660, 0.2006747343758437)]


In [55]:
idx, sim_list = top_10_similarity(39)
print("---"*10)
print(df.iloc[idx]['content'][:300])
print("---"*10)
print(df.iloc[sim_list[1][0]]['content'][:300])
print("---"*10)
print(df.iloc[sim_list[2][0]]['content'][:300])

39
[(684, 1.0000000000000002), (613, 0.8786286888243482), (654, 0.8740214852032844), (578, 0.8333858225582296), (1205, 0.3461189150191075), (1392, 0.3287127463565469), (1084, 0.3265454766455683), (2179, 0.3175055397353149), (1380, 0.3075326011931542), (1536, 0.3072942894431267)]
------------------------------
경인여자대학교 간호학과와 가톨릭대학교 부천성모병원 간호부는 14일 경인여대 20주년기념관 국제회의실과 간호실습실에서 프리셉터 간호사 워크숍을 실시했다. 사진은 경인여대 캠퍼스 전경.

경인여자대학교 간호학과와 가톨릭대학교 부천성모병원 간호부는 14일 경인여대 20주년기념관 국제회의실과 간호실습실에서 프리셉터 간호사 워크숍을 실시했다고 15일 밝혔다.프리셉터는 신규 간호사를 1대1로 맡아 병원직무를 집중적으로 교육시키는 선배간호사들을 일컫는다.이번 워크숍은 정진옥 경인여대 간호학과 학과장, 최영숙 부천성모병원 간호부장을 비롯 간호학과
------------------------------
경인여자대학교(총장 류화선) 간호학과와 가톨릭대학교 부천성모병원 간호부는 지난 14일 경인여자대학 20주년기념관에서 '프리셉터 간호사 워크숍'을 진행했다.이번 행사는 프리셉터들의 리더쉽, 코칭스킬, 감성커뮤니케이션 등을 학생들의 전반적인 교육역량을 향상시키기 위해 마련됐다. 프리셉터는 신규 간호사를 1대1로 맡아 병원직무를 집중적으로 교육시키는 선배 간호사들을 말한다.이번 워크숍은 프리셉터의 실무코칭, 선배 프리셉터와 만남, 기본간호기술교육 등 다양한 간호 프로그램으로 진행됐다.행사에 참석한 프리셉터 간호사들은 “산·학 연계로 진행
------------------------------
​​​​​​​경인여대 간호학과 학생들과 가톨릭